In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob

In [10]:
tf.test.is_built_with_cuda()

True

In [11]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [12]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [13]:
# function needed for angle wrapping
def wrap_angles(a):
    ad = np.remainder(a,2*np.pi)
    ad[ad > np.pi] -= 2*np.pi
    ad[ad < -np.pi] += 2*np.pi
    return ad

In [14]:
building1_filenames = set(glob.glob("datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("datasets/building1/train/*.feather"))
test1_filenames = building1_filenames - training_filenames

In [15]:
import random
seed=10
random.seed(seed)
np.random.seed(seed)

In [16]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)

training_data.head()

,index,stencilAccX,stencilAccY,stencilAccZ,stencilGyroX,stencilGyroY,stencilGyroZ,iphoneOrientW,iphoneOrientX,iphoneOrientY,...,iphoneMagZ,iphoneAccX,iphoneAccY,iphoneAccZ,iphoneGyroX,iphoneGyroY,iphoneGyroZ,processedPosX,processedPosY,processedPosZ
0,0,-0.271240,0.017265,9.814681,0.005063,0.003404,0.000095,0.999879,0.004516,0.014719,...,-587.832397,-0.009735,-0.033218,-1.024872,-0.004423,-0.011610,-0.014319,0.007579,0.011608,0.004128
1,2,-0.223503,-0.000951,9.789928,0.005674,0.016008,0.000641,0.999879,0.004492,0.014767,...,-587.832432,-0.010498,-0.033798,-1.022766,-0.004423,-0.011609,-0.014309,0.007499,0.011634,0.003860
2,4,-0.264626,0.018167,9.815163,0.011305,0.009493,-0.010202,0.999878,0.004516,0.014824,...,-588.256104,-0.012772,-0.034927,-1.026323,-0.007207,-0.007093,-0.014308,0.007416,0.011658,0.003587
3,6,-0.259907,0.037563,9.826657,0.009052,0.001445,0.000408,0.999878,0.004530,0.014836,...,-588.256170,-0.013976,-0.034714,-1.034287,-0.010574,-0.002152,-0.006783,0.007329,0.011679,0.003311
4,8,-0.244414,0.017294,9.818577,0.003527,0.006615,-0.001128,0.999878,0.004523,0.014838,...,-588.523621,-0.009826,-0.033874,-1.034314,-0.015048,-0.004564,-0.007740,0.007241,0.011698,0.003030


In [17]:
checkpoints_path = os.path.join(os.getcwd(),'datasets/CheckPoints_Model_OrientNet')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_vloss_{val_loss:.4f}.hdf5')
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [18]:
orient_learning_rate = 0.0005 # convergence  within  20  epochs \ref{IDOL}

In [19]:
# Loss Function
class OreintLoss(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # custom loss L_orient = 0.5*(q [-] \hat{q})^T *Sig^{-1}* 
    W = 25
    ws = tf.constant([W,W,1,1,1,1],dtype=tf.float32)
    return tf.sqrt(tf.reduce_mean(ws*(y_pred - y_true)**2, axis=-1))

In [20]:
class OreintNet():
    def build_model(self):
        # Building Network
        #1. Define inputs
        Acc_input = Input(shape=(100,3) , name = 'Acc_input')    
        Gyro_input = Input(shape=(100,3) , name = 'Gyro_input')
        Mag_input = Input(shape=(100,3) , name = 'Mag_input')
        
        MergedLayer = Concatenate()([Acc_input , Gyro_input, Mag_input])
        
        #2. LSTM
        LSTM1 = (LSTM(100, return_sequences=True))(MergedLayer)
        LSTM2 = (LSTM(100))(LSTM1)
        
        #3. Fully-Connected (Sigma)
        Dense1 = Dense(units=100, activation='tanh')(LSTM2)
        Dense2 = Dense(units=16, activation='tanh')(Dense1)
        sigma_output = Dense(units=6, activation='linear')(Dense2) # Sigma Outputs
        
        #4. Fully-Connected (Theta)
        Dense3 = Dense(units=100, activation='tanh')(LSTM2)
        Dense4 = Dense(units=16, activation='tanh')(Dense3)
        theta_output = Dense(units=6, activation='linear')(Dense4) # Theta Outputs
        
        #5. Define and compile The model
        Network = Model([Acc_input,Gyro_input, Mag_input] ,  [theta_output , sigma_output])
        Network.compile(loss=OreintLoss(), optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate) ,metrics=['rmse'])
        return Network
        

In [21]:
network_builder = OreintNet()
orientation_network = network_builder.build_model()

2021-10-22 10:00:19.410767: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-22 10:00:19.411350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 10:00:19.411513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 10:00:19.411625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [22]:
orientation_network.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Acc_input (InputLayer)          [(None, 100, 3)]     0                                            
__________________________________________________________________________________________________
Gyro_input (InputLayer)         [(None, 100, 3)]     0                                            
__________________________________________________________________________________________________
Mag_input (InputLayer)          [(None, 100, 3)]     0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 100, 9)       0           Acc_input[0][0]                  
                                                                 Gyro_input[0][0]             

In [23]:
history = orientation_network.fit(
    batch_size=64
)

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [ ]:
pos_learning_rate = 0.001